In [1]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import random
import json

In [2]:
MACRO_ECON_PATH = '/u/ayushc/projects/GradABM/MacroEcon'
STEP2_INPUT_POP_PATH = '/u/ayushc/projects/GradABM/MacroEcon/simulator_data/NYC/output_pop_data/NYC_POP.pkl'
STEP2_SAVE_ROOT = '/u/ayushc/projects/GradABM/MacroEcon/tmp/'

In [3]:
from census_loader import CensusDataLoader
# import sys
# sys.path.append(MACRO_ECON_PATH)

/u/ayushc/agent_torch_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-27 02:55:51,893	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
pop_data_df = pd.read_pickle(STEP2_INPUT_POP_PATH)

print(pop_data_df['age_gender']['area'][:10])

0    BK0101
1    BK0101
2    BK0102
3    BK0102
4    BK0103
5    BK0103
6    BK0104
7    BK0104
8    BK0201
9    BK0201
Name: area, dtype: object


In [5]:
area_selector = pop_data_df['age_gender']['area'][:1]
print(area_selector)
geo_mapping = None

loader = CensusDataLoader(n_cpu=8, use_parallel=False, area_list=area_selector, geo_mapping=geo_mapping)

0    BK0101
Name: area, dtype: object


In [6]:
if not os.path.exists(STEP2_SAVE_ROOT):
    os.makedirs(STEP2_SAVE_ROOT)

# Create Synthetic Agent Population

In [7]:
save_population_data_path = os.path.join(STEP2_SAVE_ROOT, 'population_data.pkl')
loader.generate_basepop(input_data=pop_data_df, save_path=save_population_data_path)

INFO:process.base_pop:Processing: 0/1
/u/ayushc/projects/GradABM/MacroEcon/simulator_data/step2/process/base_pop.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ethnicity_data['probability'] = ethnicity_data.apply(lambda row: row['count'] / total_population[row['area']], axis=1)
/u/ayushc/projects/GradABM/MacroEcon/simulator_data/step2/process/base_pop.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ethnicity_data['probability'] = ethnicity_data.apply(lambda row: row['count'] / total_population[ro

In [8]:
loader.population_df.head()

,area,age,gender,ethnicity
0,BK0101,20t29,female,white
1,BK0101,20t29,female,white
2,BK0101,20t29,male,white
3,BK0101,20t29,female,white
4,BK0101,20t29,female,white


In [9]:
loader.population_df.shape

(37518, 4)

In [10]:
loader.population_df['area'].unique()

array(['BK0101'], dtype=object)

## Create Synthetic Mobility Networks

In [11]:
num_steps = 2
mobility_mapping_path = './mobility_mapping.json'

with open(mobility_mapping_path) as file:
    # Load the JSON data from the file
    mobility_mapping = json.load(file)
    
save_mobility_network_dir = os.path.join(STEP2_SAVE_ROOT, 'mobility_networks')
    
loader.generate_mobility_networks(num_steps, mobility_mapping, save_mobility_network_dir)

## Testing Mobility Networks

In [ ]:
num_agents = loader.population_df.shape[0]
agents_ages = list(loader.population_df['age'].unique())

num_steps = 30
network_save_path = './mobility_networks_dir'
if not os.path.exists(network_save_path):
    os.makedirs(network_save_path)
    
agents_ages


In [ ]:
age_df = loader.population_df['age']
age_df.shape

In [ ]:
mobility_df = pd.read_csv('random_network.csv', index_col=0)

mobility_map = {
        a: {'mu': mobility_df.loc[a, 'mu'], 'sigma': mobility_df.loc[a, 'mu']}
        for a in mobility_df.index.to_list()
    }

In [ ]:
mobility_map

In [ ]:
agent_interactions = [get_agent_interactions(age_mapping_dict[age], mobility_map) for age in age_df]

In [ ]:
agent_interactions[:10]

In [ ]:
def get_agent_interactions(age, mobility_map):
    mean, sd = mobility_map[age]['mu'], mobility_map[age]['sigma']
    
    p = mean / (sd*sd)
    n = mean * mean / (sd * sd - mean)
    num_interactions = nbinom.rvs(n, p)
    
    return num_interactions

In [ ]:
def create_random_networks(age_df, num_steps, mobility_map, age_mapping_dict, save_path=None):
    num_agents = age_df.shape[0]
    
    agents_random_interactions = [get_agent_interactions(age_mapping_dict[age], mobility_map) for age in age_df]
    
    interactions_list = []
    mobility_network_paths = []
    
    for t in range(num_steps):
        for agent_id in range(num_agents):
            interactions_list.extend([agent_id]*agents_random_interactions[agent_id])
        random.shuffle(interactions_list)
        
        edges_list = [(interactions_list[i], interactions_list[i+1]) for i in range(len(interactions_list)-1)]
        G = nx.Graph()
        G.add_edges_from(edges_list)
        
        if save_path is not None:
            outfile = os.path.join(save_path, '{}.csv'.format(t))
            nx.write_edgelist(G, outfile, delimiter = ',', data = False)
            mobility_network_paths.append(outline)

    return mobility_network_paths


In [ ]:
## Execution file for mobility_mapping

age_df = loader.population_df['age']
num_steps = 3

mobility_df = pd.read_csv('random_network.csv', index_col=0)
mobility_map = {
        a: {'mu': mobility_df.loc[a, 'mu'], 'sigma': mobility_df.loc[a, 'mu']}
        for a in mobility_df.index.to_list()
    }

age_mapping_dict = {'U19': 'CHILD', '20t29': 'ADULT', '30t39': 'ADULT', '40t49': 'ADULT', '50t64': 'ADULT', '65A': 'ELDERLY'}

save_path = './mobility_networks_dir/'

create_random_networks(age_df, num_steps, mobility_map, age_mapping_dict, save_path=save_path)

In [ ]:
mobility_map

In [ ]:
import json

json_file_path = 'mobility_mapping.json'
with open(json_file_path) as file:
    # Load the JSON data from the file
    data = json.load(file)

## Generate demographic data files

In [ ]:
sys.path.append('/Users/shashankkumar/Documents/GitHub/MacroEcon/AgentTorch/AgentTorch')
from utils import preprocess_data

preprocess_data(df_path="/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/population_data.pkl", output_dir="/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC")

## Generate Household Population

In [ ]:
household_data_path = '../../housing_v2.pkl'
household_data = np.load(household_data_path, allow_pickle=True)
household_mapping = {'adult_list': ['20t29','30t39', '40t49', '50t64', '65A'], 'children_list': ['U19']}

In [ ]:
geo_address_data =None # Geo address data from excel file if exists (long, lat). The location (for every entity) for each geographic area (NTA)
save_household_path = os.path.join(SAVE_ROOT, 'base_household.pkl')

loader.generate_household(household_data, household_mapping, save_path=save_household_path)